# Lecture: Witness Sets

In [1]:
using HomotopyContinuation

So far, we've seen that numerical methods are very good at solving **zero-dimensional polynomial systems**

**Question**: What do we do if we are interested in a **positive-dimensional** set of solutions?

**Answer**: Slice it with enough hyperplanes until we have a zero-dimensional solution set.



<img src="PictureWitness1.png" width="300">

The data in such a *slice* is called a **witness set**.

A witness set for an irreducible variety $X \subset \mathbb{C}^n$ is a triple 
$$W=(F,L,S)$$
where 
    
- $\color{blue}{F}$ is a polynomial system for which $X$ is a component, 
- $\color{green}{L}$ is a generic affine linear space of complementary dimension
- $\color{red}{S} = X \cap L$



<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
How is this a good representation of the variety $X$?
</p>


Given a witness set $W=(F,L,S)$ for $X$...

Certain information about $X$ become obvious:
- dimension (equals the codimension of $L$)
- degree (equals the number of points in $S$)

Certain algorithms become almost trivial

- Membership: is $p$ contained in $X$?

- Sampling: find $100000$ points on $X$

- Find a (pseudo) witness set for a projection of $X$

One major advantage (compared to symbolic representations) is that they can be computed fast. 

One major disadvantage is that it can be difficult to prove things (but not impossible)

<hr style="border:1px solid gray"> </hr>

## Constructing a witness set using homotopycontinuation.jl

In [2]:
@var x y z
TwistedCubic=System([y^2-x*z,x*y-z,x^2-y])

System of length 3
 3 variables: x, y, z

 -x*z + y^2
 -z + x*y
 -y + x^2

In [3]:
#Try the following without codimension: you get an error because it expects the dimension to be zero.
W_twisted=witness_set(TwistedCubic; codim=2)

Tracking 5 paths... 100%|███████████████████████████████| Time: 0:00:08
  # paths tracked:                  5
  # non-singular solutions (real):  3 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (0)


Witness set for dimension 1 of degree 3

In [4]:
#Degree of the twisted cubic
degree(W_twisted)

3

In [5]:
#Dimension of the twisted cubic
dim(W_twisted)

1

In [6]:
#Generic linear space used to slice the twisted cubic
W_twisted.L

2-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.1831417641570392 + 0.081102212943236im -0.10672217795974322 - 0.7463036812511545im -0.13540631545481352 - 0.6108910271845651im]
b:
Complex{Float64}[-0.2313869318216478 - 0.08994162549643306im]

In [7]:
#Witness points
solutions(W_twisted)

3-element Array{Array{Complex{Float64},1},1}:
 [0.4431396138501076 - 0.1317374677130209im, 0.17901795696378325 - 0.11675618114387824im, 0.0639487846774288 - 0.07532266135227876im]
 [-0.3070327111783467 + 0.5200813468544931im, -0.17621552161245757 - 0.3193639719160423im, 0.22019917400327516 + 0.006408780333193877im]
 [-1.3374703446145122 - 0.47993144017293227im, 1.5584927354567966 + 1.2837881373588613im, -1.468307526331051 - 2.4649982255122116im]

## Some simple algorithms involving witness sets

### Moving the linear space (this is the fundamental operation)

In [8]:
L2 = LinearSubspace([1,2,3]',[5])

2-dim. affine linear subspace {x | Ax=b} with eltype Float64:
A:
[-0.2672612419124243 -0.5345224838248488 -0.8017837257372732]
b:
[-1.3363062095621219]

In [9]:
W_twisted2=witness_set(W_twisted,L2)

Tracking 3 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  3
  # non-singular solutions (real):  3 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (1)


Witness set for dimension 1 of degree 3

In [10]:
solutions(W_twisted2)

3-element Array{Array{Complex{Float64},1},1}:
 [0.9241438381955056 - 1.3775324423698682e-40im, 0.8540418336747209 + 1.8367099231598242e-40im, 0.7892574981516842 + 2.2958874039497803e-41im]
 [-0.7954052524310862 + 1.0820358328370792im, -0.5381320279484715 - 1.721313969514515im, 2.290556436109343 + 0.786864035397317im]
 [-0.7954052524310862 - 1.082035832837079im, -0.5381320279484714 + 1.7213139695145148im, 2.290556436109343 - 0.7868640353973171im]

### Sampling

In [11]:
function sample(W::WitnessSet)
    n=length(solutions(W)[1]) #number of variables
    randomLinear = LinearSubspace(randn(ComplexF64,dim(W),n),randn(ComplexF64,dim(W)))
    newWitness=witness_set(W,randomLinear)
    s=solutions(newWitness)[1]
    return s
end

sample (generic function with 1 method)

In [12]:
MySamples=[sample(W_twisted) for i in 1:100]

100-element Array{Array{Complex{Float64},1},1}:
 [1.4659465353363361 + 0.2936973130876942im, 2.0627411327496765 + 0.8610891171169929im, 2.7709686568238214 + 1.868132136137527im]
 [0.9817013309365323 + 0.811592453899928im, 0.3050551919352523 + 1.5934827843432111im, -0.9937855152604598 + 1.8119046620118264im]
 [0.22901897494200477 + 1.549831727329766im, -2.3495286921544793 + 0.7098817470513193im, -1.638283906906466 - 3.47879772133289im]
 [0.34454494720722845 + 1.5076197497784738im, -2.154206089276076 + 1.0388855341919983im, -2.308465172410132 - 2.889780883753127im]
 [1.236758724393006 + 0.6606794100505745im, 1.0930748594974402 + 1.6342020488137443im, 0.27218622338441384 + 2.743285694605207im]
 [1.6556417736737985 - 2.12198966851813im, -1.7616904705639616 - 7.026509477005665im, -17.826908851299496 - 7.895093635581844im]
 [0.5353617881800264 - 0.4879282355863325im, 0.048538281160923916 - 0.5224362654140483im, -0.2289258641922924 - 0.3033756111474028im]
 [0.6317904182926204 - 0.097880986727

### Membership

In [13]:
function membership(W::WitnessSet,p)
    n=length(solutions(W)[1]) #number of variables
    A=randn(ComplexF64,dim(W),n)
    randomLinearThroughP = LinearSubspace(A,A*p)
    newWitness=witness_set(W,randomLinearThroughP)
    check=findfirst(x->x≈p,solutions(newWitness))
    if check==nothing
        return false
    else
        return true
    end
end
    

membership (generic function with 1 method)

In [14]:
membership(W_twisted,[2,4,8])

true

In [15]:
membership(W_twisted,[2,5,8])

false

In [16]:
[membership(W_twisted,s) for s in MySamples]

100-element Array{Bool,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

## Pseudo witness sets

Both the sampling and membership algorithms (and most other algorithms involving witness sets) depend on one thing:


**the ability to move the linear space of the witness set and track the witness points**

Observation: we can do this for projections trivially

**Definition:** A pseudo witness set for an (irreducible) variety $X\subset \mathbb{C}^n$ is a quadruple 
$$(F,\pi,\pi^{-1}(L),S)$$
where 
- $\pi:\mathbb{C}^N \to \mathbb{C}^n$ (really this works with any map by factoring through the graph)
- $X$ is a component of $\pi(\mathcal{V}(F))$ of the same dimension as $\mathcal{V}(F)$
- $L$ is a linear space of complementary dimension (to $X$) in $\mathbb{C}^n$
- $S = \pi^{-1}(L) \cap \mathcal V(F)$ (note! this is in $\mathbb{C}^N$)

<img src="PictureWitness2.png" width="300">

As the picture above shows, $\pi^{-1}(L)$ may not be generic for $\mathcal{V}(F)$ even though $L$ is generic for $X$


**Warning** be careful about the following situation

<img src="PictureWitness3.png" width="200">

In [17]:
PseudoWitness_Parabola=witness_set(W_twisted,LinearSubspace([3,6,0]',[1]))

Witness set for dimension 1 of degree 2

In [18]:
ProjectedSolutions=[s[1:2] for s in solutions(PseudoWitness_Parabola)]

2-element Array{Array{Complex{Float64},1},1}:
 [0.22871355387816908 + 7.52316384526264e-37im, 0.05230988972758215 - 7.52316384526264e-37im]
 [-0.728713553878169 + 0.0im, 0.5310234436057513 - 2.2420775429197073e-44im]

In [19]:
Witness_Parabola=witness_set(System([y-x^2])) #computes a witness set for the parabola
Witness_Parabola2=witness_set(Witness_Parabola,LinearSubspace([3,6]',[1])) #moves it to the space we used above

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:01
  # paths tracked:                  2
  # non-singular solutions (real):  2 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (2)


Witness set for dimension 1 of degree 2

In [20]:
solutions(Witness_Parabola2)

2-element Array{Array{Complex{Float64},1},1}:
 [0.22871355387816908 + 0.0im, 0.05230988972758215 + 0.0im]
 [-0.728713553878169 + 0.0im, 0.5310234436057513 - 1.925929944387236e-34im]

Membership/sampling can be easily coded for these pseudo-witness sets as well

## Witness supersets, monodromy, the trace test, and irreducible decomposition

In [21]:
#If we only take two of the generators, we get a complete intersection of the union of a cubic and a line
CompleteIntersection=System([y^2-x*z,x*y-z])
W_complete=witness_set(CompleteIntersection)

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  4
  # non-singular solutions (real):  4 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (0)


Witness set for dimension 1 of degree 4


<img src="PictureWitness4.png" width="200">

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Warning: W_complete is not a witness set for the twisted cubic.
</p>

A **witness superset** for $\mathcal{V}(f_1,\ldots,f_k)$ in codimension $m$ is the triple $W=(F,L,S)$
    
- $\color{blue}{F}$ is a polynomial system
- $\color{green}{L}$ is a generic affine linear space of dimension $m$
- $\color{red}{S} = X \cap L$


Let $X \subseteq \mathbb{C}^n$ be an (equidimensional) variety of codimension $m$.

We have a branched cover 


$$Z = \{(x,L) \in \mathbb{C}^n \times \textrm{Gr}(m+1,n+1) \mid x \in X \cap L \} \subset \mathbb{C}^n \times \textrm{Gr}(m+1,n+1)$$
$$\,\,\,\, \downarrow \pi $$
$$\textrm{Gr}(m+1,n+1)$$



**Theorem**: If $X$ is irreducible then the monodromy group of $\pi$ is the full symmetric group.

**Corollary**: If $X=X_1,\ldots,X_k$ is equidimensional, then the monodromy group of $\pi$  is the product of symmetric groups $\mathfrak{S}_{\textrm{deg}(X_1)} \times \cdots \times \mathfrak{S}_{\textrm{deg}(X_k)}$

Let's try to compute the witness points using monodromy

In [22]:
M=monodromy_solve(CompleteIntersection; dim=1) 

MonodromyResult
• return_code → :success
• 3 solutions
• 12 tracked loops
• random_seed → 0xda3d2f5c
• trace → 1.0033888943804392e-16

In [23]:
#Trying this many times will give us either 1 or 3 solutions (possibly 2 if the heuristic stop didn't work)
[length(solutions(monodromy_solve(CompleteIntersection;dim=1))) for i in 1:5]

5-element Array{Int64,1}:
 2
 1
 1
 1
 1

**Monodromy**: allows us to find points on the same component

**The trace test**: tells us when we've found all points on the same component


The **trace** of a subset of points in $\mathbb{C}^n$ is their coordinate-wise average.

**Lemma**: Let $X$ be irreducible. The trace of a subset of witness points of $X$ moves linearly if and only if the subset is not proper.


<img src="TraceExample1.png" width="600">

Now let's try moving a witness set and verifying this ourselves

In [24]:
W_complete

Witness set for dimension 1 of degree 4

In [25]:
A=randn(ComplexF64,1,3)
b=rand(ComplexF64)
L₁ = LinearSubspace(A,[b])
L₂ = LinearSubspace(A,[b+1])
L₃ = LinearSubspace(A,[b+2]);

In [26]:
trace₁=sum(solutions(witness_set(W_complete,L₁)))
println("First trace: \n",trace₁)
trace₂=sum(solutions(witness_set(W_complete,L₂)))
println("Second trace: \n",trace₂)
trace₃=sum(solutions(witness_set(W_complete,L₃)))
println("Third trace: \n",trace₃)

First trace: 
Complex{Float64}[2.092466405094321 + 0.5186410304722054im, -2.8199096466209395 + 2.9441001500950543im, -5.635685780733304 - 1.3831831635779048im]
Second trace: 
Complex{Float64}[1.8863587415914775 - 0.5017011275440446im, -2.81990964662094 + 2.944100150095054im, -8.120198589813198 - 7.686613750606742im]
Third trace: 
Complex{Float64}[1.680251078088634 - 1.5220432855602946im, -2.8199096466209412 + 2.944100150095054im, -10.604711398893091 - 13.990044337635581im]


In [27]:
# here we check trace2 is the midpoint of trace1 and trace3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
                    0.0 + 0.0im
 -8.881784197001252e-16 + 8.881784197001252e-16im
                    0.0 - 1.7763568394002505e-15im

In [28]:
trace_test(W_complete)

3.4122811639178395e-17

We can find out which solution is the one on the line by checking traces of subsets

In [29]:
I=[1,3,4]
trace₁=sum(solutions(witness_set(W_complete,L₁))[I])
trace₂=sum(solutions(witness_set(W_complete,L₂))[I])
trace₃=sum(solutions(witness_set(W_complete,L₃))[I])
# here we check trace2 is the midpoint of trace1 and trace 3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 0.12328802187978116 - 0.31302948262619457im
 0.04782484636836237 - 0.31544099244541535im
 0.16522081212753648 + 0.3018551143440167im

In [30]:
[membership(W_twisted,s) for s in solutions(W_complete)]

4-element Array{Bool,1}:
 0
 1
 1
 1

### Numerical irreducible decomposition (of equidimensional varieties)

One of the most important algorithms in numerical algebraic geometry is taking a polynomial systme $F$ and computing a witness set for every irreducible component of $\mathcal V(F)$. This is called a **numerical irreducible decomposition**. We will go through how it works for equidimensional varieties

#### Step 1: compute a witness superset 

$\mathcal V(F) \to (F,L,S)$

#### Step 2: 

Using monodromy, identify solutions as belonging to the same component until the trace test passes.

$p \in S \to S_p=\{q \in S \mid q \text{ was found via monodromy starting from } p\}$

#### Step 3: 

If $S \backslash \bigcup_{p \text{ done in step 2}} S_p \neq \emptyset$ pick some $q$ not identified and repeat step 2

#### Output: 

Set $W_p = (F,L,S_p)$ and output the union of all $W_p$

In [49]:
using Bertini

In [48]:
F=System([(x^2+y^2-2)*((x-1)^2+y^2-3)*(x-y+1),(x^2+y^2-2)*((x-1)^2+y^2-3)*(x+y-6)])
bertini(F; TrackType=1)

File path: /tmp/jl_rVZDYS

 Bertini(TM) v1.5.1
  (August 29, 2016)

 D.J. Bates, J.D. Hauenstein,
 A.J. Sommese, C.W. Wampler

(using GMP v6.1.2, MPFR v3.1.4)



NOTE: You have requested to use adaptive path tracking.  Please make sure that you have
setup the following tolerances appropriately:
CoeffBound: 1.520470000000e+02, DegreeBound: 5.000000000000e+00
AMPSafetyDigits1: 1, AMPSafetyDigits2: 1, AMPMaxPrec: 1024


Tracking regeneration codim 1 of 2: 5 paths to track.
Tracking path 0 of 5

Sorting codimension 1 of 2: 5 paths to sort.
Sorting 0 of 5

Preparing regeneration codim 2 of 2: 4 witness points to move.
Moving 0 of 4

Tracking regeneration codim 2 of 2: 5 paths to track.
Tracking path 0 of 5

Sorting codimension 2 of 2: 5 paths to sort.
Sorting 0 of 5


************ Regenerative Cascade Summary ************

NOTE: nonsingular vs singular is based on rank deficiency and identical endpoints

|codim|   paths   |witness superset| nonsingular | singular |nonsolutions| inf endpoint

LoadError: ArgumentError: Cannot open 'finite_solutions': not a file

### Other things you can do with witness sets

Given a witness set $W$ for $X$ you can also...

- find a witness set for $X \cap \mathcal V(f)$
- recover equations for $X$
- compute the Newton polytope of $X$ (when $X$ is a hypersurface)
- describe the real points of $X$
- much more...